# **FIONA HARIA 60009220048 D1-2 D040**

# Formulas , Literals and User Selected Bits

In [1]:
# F1: (A V ~B) ^ (B V ~C) ^ (~B) ^ (~C V E) ^ (A V C) ^ (~C V ~D)
F1 = [
    [1, -2],
    [2, -3],
    [-2],
    [-3, 5],
    [1, 3],
    [-3, -4]
]

# F2: (A V B) ^ (A ^ ~C) ^ (B ^ D) ^ (A V ~E)
F2 = [
    [1, 2],
    [1, -3],
    [2, 4],
    [1, -5]
]

user_selected_bits = [0, 2, 4]

# Clause Evaluation and Heuristic

In [2]:
import random
def evaluate_clause(clause, assignment):
    for literal in clause:
        if literal > 0 and assignment[literal - 1]:
            return True
        if literal < 0 and not assignment[-literal - 1]:
            return True
    return False

def evaluate_formula(formula, assignment):
    return all(evaluate_clause(clause, assignment) for clause in formula)

def heuristic(formula, assignment):
    satisfied_clauses = sum(evaluate_clause(clause, assignment) for clause in formula)
    return satisfied_clauses

# MoveGen and Neighbour function

In [3]:
def neighbors(assignment, selected_bits):
    neighborhood = []
    for i in selected_bits:
        new_assignment = assignment[:]
        new_assignment[i] = not new_assignment[i]
        neighborhood.append(new_assignment)
        print(f"Generated neighbor by flipping bit {i}: {new_assignment}")
    return neighborhood

def move_generation(formula, assignment, selected_bits=None):
    if selected_bits is None:
        selected_bits = list(range(len(assignment)))

    best_assignment = assignment
    best_heuristic_value = heuristic(formula, assignment)
    print(f"Initial assignment: {assignment}, Heuristic value: {best_heuristic_value}")

    for neighbor in neighbors(assignment, selected_bits):
        current_heuristic_value = heuristic(formula, neighbor)
        print(f"Evaluating neighbor: {neighbor}, Heuristic value: {current_heuristic_value}")
        if current_heuristic_value > best_heuristic_value:
            best_assignment = neighbor
            best_heuristic_value = current_heuristic_value
            print(f"New best assignment: {best_assignment}, Heuristic value: {best_heuristic_value}")

    return best_assignment, best_heuristic_value

# Variable Neighbourhood Descent

In [4]:
def VND(formula, max_iter=1000, selected_bits=None):
    n = len(set(abs(lit) for clause in formula for lit in clause))
    current_assignment = [random.choice([True, False]) for _ in range(n)]
    best_assignment, best_heuristic_value = current_assignment, heuristic(formula, current_assignment)
    for _ in range(max_iter):
        new_assignment, new_heuristic_value = move_generation(formula, best_assignment, selected_bits)
        if new_heuristic_value > best_heuristic_value:
            best_assignment = new_assignment
            best_heuristic_value = new_heuristic_value
        else:
            break

    return best_assignment, evaluate_formula(formula, best_assignment)

# Testing the MoveGen function


In [5]:
print("Testing move generation for F1:")
solution_F1, is_satisfied_F1 = VND(F1, selected_bits=user_selected_bits)
print(f"Final solution for F1: {solution_F1}, Satisfied: {is_satisfied_F1}\n")

Testing move generation for F1:
Initial assignment: [False, False, False, True, True], Heuristic value: 5
Generated neighbor by flipping bit 0: [True, False, False, True, True]
Generated neighbor by flipping bit 2: [False, False, True, True, True]
Generated neighbor by flipping bit 4: [False, False, False, True, False]
Evaluating neighbor: [True, False, False, True, True], Heuristic value: 6
New best assignment: [True, False, False, True, True], Heuristic value: 6
Evaluating neighbor: [False, False, True, True, True], Heuristic value: 4
Evaluating neighbor: [False, False, False, True, False], Heuristic value: 5
Initial assignment: [True, False, False, True, True], Heuristic value: 6
Generated neighbor by flipping bit 0: [False, False, False, True, True]
Generated neighbor by flipping bit 2: [True, False, True, True, True]
Generated neighbor by flipping bit 4: [True, False, False, True, False]
Evaluating neighbor: [False, False, False, True, True], Heuristic value: 5
Evaluating neighbor

In [6]:
print("Testing move generation for F2:")
solution_F2, is_satisfied_F2 = VND(F2, selected_bits=user_selected_bits)
print(f"Final solution for F2: {solution_F2}, Satisfied: {is_satisfied_F2}")

Testing move generation for F2:
Initial assignment: [False, False, False, False, True], Heuristic value: 1
Generated neighbor by flipping bit 0: [True, False, False, False, True]
Generated neighbor by flipping bit 2: [False, False, True, False, True]
Generated neighbor by flipping bit 4: [False, False, False, False, False]
Evaluating neighbor: [True, False, False, False, True], Heuristic value: 3
New best assignment: [True, False, False, False, True], Heuristic value: 3
Evaluating neighbor: [False, False, True, False, True], Heuristic value: 0
Evaluating neighbor: [False, False, False, False, False], Heuristic value: 2
Initial assignment: [True, False, False, False, True], Heuristic value: 3
Generated neighbor by flipping bit 0: [False, False, False, False, True]
Generated neighbor by flipping bit 2: [True, False, True, False, True]
Generated neighbor by flipping bit 4: [True, False, False, False, False]
Evaluating neighbor: [False, False, False, False, True], Heuristic value: 1
Evalua

In [7]:
print(f"Solution for F1: {solution_F1}, Satisfied: {is_satisfied_F1}")
print(f"Solution for F2: {solution_F2}, Satisfied: {is_satisfied_F2}")

Solution for F1: [True, False, False, True, True], Satisfied: True
Solution for F2: [True, False, False, False, True], Satisfied: False


# Printing the state space tree

In [8]:
class TreeNode:
    def __init__(self, assignment, heuristic_value):
        self.assignment = assignment
        self.heuristic_value = heuristic_value
        self.children = []

    def add_child(self, child_node):
        self.children.append(child_node)


def build_state_space_tree(formula, max_iter=1000, selected_bits=None):
    n = len(set(abs(lit) for clause in formula for lit in clause))
    current_assignment = [random.choice([True, False]) for _ in range(n)]
    root = TreeNode(current_assignment, heuristic(formula, current_assignment))

    best_assignment, best_heuristic_value = current_assignment, root.heuristic_value

    current_node = root
    for _ in range(max_iter):
        new_assignment, new_heuristic_value = move_generation(formula, best_assignment, selected_bits)

        new_node = TreeNode(new_assignment, new_heuristic_value)
        current_node.add_child(new_node)

        if new_heuristic_value > best_heuristic_value:
            best_assignment = new_assignment
            best_heuristic_value = new_heuristic_value
            current_node = new_node
        else:
            break

    return root


def print_state_space_tree(node, level=0):
    indent = "  " * level
    print(f"{indent}Assignment: {node.assignment}, Heuristic: {node.heuristic_value}")

    for child in node.children:
        print_state_space_tree(child, level + 1)

root_F1 = build_state_space_tree(F1, selected_bits=user_selected_bits)
root_F2 = build_state_space_tree(F2, selected_bits=user_selected_bits)

print("State Space Tree for F1:")
print_state_space_tree(root_F1)

print("\nState Space Tree for F2:")
print_state_space_tree(root_F2)

Initial assignment: [False, False, True, False, True], Heuristic value: 5
Generated neighbor by flipping bit 0: [True, False, True, False, True]
Generated neighbor by flipping bit 2: [False, False, False, False, True]
Generated neighbor by flipping bit 4: [False, False, True, False, False]
Evaluating neighbor: [True, False, True, False, True], Heuristic value: 5
Evaluating neighbor: [False, False, False, False, True], Heuristic value: 5
Evaluating neighbor: [False, False, True, False, False], Heuristic value: 4
Initial assignment: [False, False, False, False, True], Heuristic value: 1
Generated neighbor by flipping bit 0: [True, False, False, False, True]
Generated neighbor by flipping bit 2: [False, False, True, False, True]
Generated neighbor by flipping bit 4: [False, False, False, False, False]
Evaluating neighbor: [True, False, False, False, True], Heuristic value: 3
New best assignment: [True, False, False, False, True], Heuristic value: 3
Evaluating neighbor: [False, False, True

# Conclusion

Variable Neighborhood Descent (VND) applied to SAT problems, as demonstrated in the above code, explores possible assignments by flipping selected bits and evaluating their quality using a heuristic function. It improves the current solution by exploring its neighbors and selecting the best one based on heuristic performance. The process terminates when no better neighbors are found, making VND an effective global search algorithm for optimizing SAT problem solutions within a given neighborhood.